in 06.1 i defined cohort with atleast 1 SpO2 reading present. This will be the missingness analysis performed on this subset of patients. 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime
%reload_ext autotime

In [2]:
#cohort import

os.chdir('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling') #use to change working directory
wd= os.getcwd() #'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling'


final_pt_df2 = pd.read_csv(wd + '/data/raw/csv/01102018_final_pt_df2_v.csv' , index_col=0)
patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 86.7 ms


In [3]:
len(patients)

12403

time: 4.01 ms


In [4]:
#reading in all of my data that is not limited to 72 hour time window between t_0 and t+72
date= '27082018' 

vaso_dose_72=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_vaso_dose_72.csv' %(date), index_col=0)
#ventsettings_72=pd.read_csv(
#    wd+'/data/raw/csv/72_hr_window/%s_ventsettings_72.csv' %(date), index_col=0)

ventcategory_df=pd.read_csv(
    wd+'/data/raw/csv/%s_ventcategory_df.csv' %(date), index_col=0)

echodata_72=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_echodata_72.csv' %(date), index_col=0)
labs_all_nosummary_72=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_labs_all_nosummary_72.csv' %(date), index_col=0)


weightfirstday_df=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_weightfirstday_df.csv' %(date), index_col=0)
heightfirstday_df=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_heightfirstday_df.csv' %(date), index_col=0)

vitals_all_nosummary_72=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_vitals_all_nosummary_72.csv' %(date), index_col=0)
uti_all_72=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_uti_all_72.csv' %(date), index_col=0)
bg_all_nosummary_72=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_bg_all_nosummary_72.csv' %(date), index_col=0)

rrt_merged_allpt_df=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_rrt_merged_allpt_df.csv' %(date), index_col=0)
gcs72_df=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_gcs72_df.csv' %(date), index_col=0)

sofa_df_72=pd.read_csv(
    wd+'/data/raw/csv/%s_sofa_df_72.csv' %(date), index_col=0)

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 14 s


In [ ]:
final_pt_df2.head()

In [ ]:
vaso_dose_72.head(20)

# task 1: % MISSINGNESS table
need to look for the amount of patients who are missing any value in any of the categories. want to aggregate all variables and % missingness and source into 1 dataframe

In [ ]:
print(
    labs_all_nosummary_72['hadm_id'].nunique(), #15142/15207 unique hadm_id/icustay_id
    labs_all_nosummary_72['subject_id'].nunique() #12744/12801 unique patients in this
)

In [ ]:
def missingness_fxn(df,name, groupby,filteron):
    
    """
    input: dataframe w/ clinical data we want to assess the % missing values on.
    output: a dataframe grouped by the different clinical variables with 
        associated % missingnss (ie how many icustays do not have this value)
        
    """

    missing_df= pd.DataFrame(
        100* (1-(df.groupby(groupby)[filteron].nunique()/ len(hadm_id)))) #number of unique icustays with data /total # icustay
    missing_df.reset_index(inplace=True)
    missing_df= missing_df.rename(index=str, columns={"label":'label', filteron:'%missingness'})
    missing_df['source']= name
    missing_df['data_label']=groupby
    return(missing_df)

#### all dataframes to do %missingness on (add x when complete)
- vaso_dose_72 : x
- ventcategory_df : x
- echodata_72 : ?
- labs_all_nosummary_72 : x
- weightfirstday_df : x
- heightfirstday_df : x
- vitals_all_nosummary_72 :x
- uti_all_72 :x
- bg_all_nosummary_72 :x
- rrt_merged_allpt_df :x
- gcs72_df : x
- sofa_df_72: 

In [ ]:
missingness_df= pd.DataFrame()

### labs x

In [ ]:
print(
    labs_all_nosummary_72['hadm_id'].nunique(), #15142/15207 unique hadm_id/icustay_id
    labs_all_nosummary_72['subject_id'].nunique() #12744/12801 unique patients in this
)

In [ ]:
#na values
labs_all_nosummary_72.loc[labs_all_nosummary_72['valuenum'].isna(),:]

In [ ]:
missingness_df = pd.concat(
    [missingness_df,
        missingness_fxn(labs_all_nosummary_72,'labs_all_nosummary_72', 'label','icustay_id')]
)

### vaso_dose x

In [ ]:
print(
    vaso_dose_72['icustay_id'].nunique(), #4777/15207 unique icustay_id/icustay_id
)

In [ ]:
vaso_dose_72.head(2)

In [ ]:
vaso_dose_72.loc[vaso_dose_72['vaso_rate'].isna(),:]

In [ ]:
missingness_df = pd.concat(
    [missingness_df,
        missingness_fxn(vaso_dose_72,'vaso_dose_72', 'label','icustay_id')]
)

### ventcategory_df x

In [ ]:
ventcategory_df.head()

In [ ]:
#need to reshape the dataframe
ventcategory_df_melt = pd.melt(ventcategory_df, id_vars=['icustay_id','t_0'], var_name='label')

print(
    ventcategory_df_melt['icustay_id'].nunique(), #15207/15207 unique icustay_id/icustay_id
)

In [ ]:
ventcategory_df_melt.head(2)

In [ ]:
#na values
ventcategory_df_melt.loc[ventcategory_df_melt['value'].isnull()]

In [ ]:
missingness_df = pd.concat(
    [missingness_df,
        missingness_fxn(ventcategory_df_melt,'ventcategory_df', 'label','icustay_id')]
)

### echodata_72

In [ ]:
echodata_72.head(2)

In [ ]:
#num of icustay_id/hadm_id
print(
    echodata_72['hadm_id'].nunique(), #5421/15207 unique hadm_id/icustay_id
)

In [ ]:
#number of na values

pd.DataFrame(
    {
    'bsa':[echodata_72.loc[echodata_72['bsa'].isna(),'hadm_id'].count()], 
    'bp': [echodata_72.loc[echodata_72['bp'].isna(),'hadm_id'].count()], 
    'bpsys': [echodata_72.loc[echodata_72['bpsys'].isna(),'hadm_id'].count()], 
    'bpdias': [echodata_72.loc[echodata_72['bpdias'].isna(),'hadm_id'].count()],
    'hr': [echodata_72.loc[echodata_72['hr'].isna(),'hadm_id'].count()],
    'test': [echodata_72.loc[echodata_72['test'].isna(),'hadm_id'].count()]
             }
            )

# NEED TO ASK ABOUT WHICH VALUES ARE MOST IMPORTANT IN THIS

In [ ]:
# missingness_df = pd.concat(
#     [missingness_df,
#         missingness_fxn(labs_all_nosummary_72,'labs_all_nosummary_72', 'label','icustay_id')]
# )

### weightfirstday_df x

In [ ]:
#delete once done debugging
weightfirstday_df=pd.read_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_weightfirstday_df.csv' %(date), index_col=0)

In [ ]:
#need to reshape the df

weightfirstday_df_melt= pd.melt(weightfirstday_df, id_vars='icustay_id', var_name='label')
#weightfirstday_df_melt

In [ ]:
print(
    weightfirstday_df_melt['icustay_id'].nunique(), #15207/15207 unique icustay_id/icustay_id
)

In [ ]:
#na values
weightfirstday_df_melt.loc[weightfirstday_df_melt['value'].isna(),'icustay_id'].count() #1522/15207 do not have any weight, 39522 total null rows.

In [ ]:
#missingness df
missingness_df = pd.concat(
    [missingness_df,
        missingness_fxn(weightfirstday_df_melt.loc[weightfirstday_df_melt['value'].notnull(),:],'weightfirstday_df', 'label','icustay_id')]
)

In [ ]:
missingness_df

### heightfirstday_df x

In [ ]:
heightfirstday_df.head(2)

In [ ]:
#need to reshape the df
heightfirstday_df_melt= pd.melt(heightfirstday_df, id_vars='icustay_id', var_name='label')

In [ ]:
print(
    heightfirstday_df_melt['icustay_id'].nunique(), #15207/15207 unique icustay_id/icustay_id
)

In [ ]:
#na values
print(
    heightfirstday_df_melt.loc[heightfirstday_df_melt['value'].isna(),'icustay_id'].count(), #22776 total null rows.
    heightfirstday_df.loc[heightfirstday_df['height'].isna(),'icustay_id'].count() #5212/15207 do not have any weight,
)

In [ ]:
#missingness df
missingness_df = pd.concat(
    [missingness_df,
        missingness_fxn(heightfirstday_df_melt.loc[heightfirstday_df_melt['value'].notnull(),:],'heightfirstday_df', 'label','icustay_id')]
)

In [ ]:
missingness_df

### vitals_all_nosummary_72 x

In [ ]:
vitals_all_nosummary_72.head(2)

In [ ]:
#na values
vitals_all_nosummary_72.loc[vitals_all_nosummary_72['vitalid'].notnull(),:]#.count() #6930 NULL values

vitals_all_nosummary_nonull= vitals_all_nosummary_72.loc[vitals_all_nosummary_72['vitalid'].notnull(),:]#.count() #6930 NULL values

#why do i have null vitalid's with actual values? need to dive into the sql.
###it looks like it may just extract a lot of values as null that are not relevant, can filter these out

In [ ]:
print(
    vitals_all_nosummary_nonull['icustay_id'].nunique(), #14714/15207 unique icustay_id/icustay_id
)

In [ ]:
vitals_all_nosummary_nonull

In [ ]:
missingness_df = pd.concat(
    [missingness_df,
        missingness_fxn(vitals_all_nosummary_nonull.rename(index=str, columns={'vitalid':'label'}),'vitals_all_nosummary_72', 'label','icustay_id')]
)

### uti_all_72 x

In [ ]:
uti_all_72=pd.read_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_uti_all_72.csv' %(date), index_col=0)

In [ ]:
uti_all_72.head(2)

In [ ]:
#uti_all_72_melt= pd.melt(uti_all_72, id_vars=['subject_id','hadm_id','itemid','charttime','t_0','delta'], var_name='label')

In [ ]:
uti_all_72_melt= pd.melt(
    uti_all_72[['hadm_id','charttime','value','valuenum','valueuom','label','fluid']],
    id_vars=['hadm_id','charttime'], var_name='label')

In [ ]:
uti_all_72_melt_notnull= uti_all_72_melt[uti_all_72_melt['value'].notnull()] #1320954

In [ ]:
uti_all_72_melt_notnull['label'].value_counts()

In [ ]:
print(
    uti_all_72_melt_notnull.nunique(), #6145/15207 unique hadm_id/hadm_id
)

In [ ]:
#na values
#uti_all_72_melt.loc[uti_all_72_melt['value'].isna(),:]#.count() #220159 na valuenum, 440318 when melted
#uti_all_72_melt[uti_all_72_melt['value'].isnull()]['hadm_id'].count() #440318

In [ ]:
#uti_all_72_melt[uti_all_72_melt['value'].notnull()]

In [ ]:
missingness_fxn(uti_all_72_melt_notnull,'uti_all_72_melt', 'label','hadm_id')

In [ ]:
missingness_df = pd.concat(
    [missingness_df,
        missingness_fxn(uti_all_72_melt_notnull,'uti_all_72_melt', 'label','hadm_id')]
)

### bg_all_nosummary_72 x

In [ ]:
bg_all_nosummary_72=pd.read_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_bg_all_nosummary_72.csv' %(date), index_col=0)

In [ ]:
#na values- removing all null values from value.
bg_all_nosummary_72= bg_all_nosummary_72.loc[bg_all_nosummary_72['value'].notnull(),:]

bg_all_nosummary_72.head(2)

In [ ]:
print(
    bg_all_nosummary_72['icustay_id'].nunique(), #11244/15207 unique icustay_id/icustay_id
)

In [ ]:
missingness_df = pd.concat(
    [missingness_df,
        missingness_fxn(bg_all_nosummary_72,'bg_all_nosummary_72', 'label','icustay_id')]
)

In [ ]:
#missingness_fxn(bg_all_nosummary_72,'bg_all_nosummary_72', 'label','icustay_id')

### rrt_merged_allpt_df x

In [ ]:
rrt_merged_allpt_df.head(10)

In [ ]:
#na values
rrt_merged_allpt_df_notnull = rrt_merged_allpt_df.loc[rrt_merged_allpt_df['rrt'].notnull(),:]

In [ ]:
print(
    rrt_merged_allpt_df_notnull['icustay_id'].nunique(), #1223/15207 unique icustay_id/icustay_id
)

In [ ]:
#missingness_fxn(rrt_merged_allpt_df_notnull.rename(index=str,columns={'rrt':"label"}),'rrt_merged_allpt_df_notnull', 'label','icustay_id')

In [ ]:
missingness_df = pd.concat(
    [missingness_df,
        missingness_fxn(rrt_merged_allpt_df_notnull.rename(index=str,columns={'rrt':"label"}),'rrt_merged_allpt_df_notnull', 'label','icustay_id')]
)

### gcs72_df x

In [ ]:
gcs72_df.head(2)

In [ ]:
#need to reshape the dataframe
gcs72_df_melt = pd.melt(gcs72_df, id_vars=['subject_id','hadm_id','icustay_id','t_0','ICU_admit','admit_plus_day','delta'], var_name='label')

In [ ]:
gcs72_df_melt_notnull= gcs72_df_melt[gcs72_df_melt['value'].notnull()] #62772 ->62650

In [ ]:
print(
    gcs72_df_melt_notnull['icustay_id'].nunique(), #10462/15207 unique icustay_id/icustay_id
)

In [ ]:
missingness_fxn(gcs72_df_melt_notnull,'gcs72_df_melt_notnull', 'label','icustay_id')

In [ ]:
missingness_df = pd.concat(
    [missingness_df,
        missingness_fxn(gcs72_df_melt_notnull,'gcs72_df_melt_notnull', 'label','icustay_id')]
)

### sofa_df_72

In [ ]:
sofa_df_72.head(2)

In [ ]:
#need to reshape the dataframe
sofa_df_melt = pd.melt(sofa_df_72, id_vars=['subject_id','hadm_id','icustay_id','t_0','day','ICU_admit','approx_charttime','floor_charttime'], var_name='label')

In [ ]:
sofa_df_melt_notnull= sofa_df_melt[sofa_df_melt['value'].notnull()] #606651 ->302459

In [ ]:
len(sofa_df_melt_notnull)

In [ ]:
print(
    sofa_df_melt_notnull['icustay_id'].nunique(), #13323/15207 unique icustay_id/icustay_id
)

In [ ]:
missingness_df = pd.concat(
    [missingness_df,
        missingness_fxn(sofa_df_melt_notnull,'sofa_df_melt_notnull', 'label','icustay_id')]
)

In [ ]:
missingness_df_v = missingness_df.sort_values('%missingness', ascending=False)

date= '01102018'

pd.DataFrame(missingness_df_v).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/interim/%s_missingness_df_v.csv' %(date))

In [ ]:
missingness_df

### visualizations

In [ ]:
"""
    get the # of filled vs missign values for each variable for our cohort.
    Args:
    -----
    df: dataframe
        name of dataframe initialized in this workbook
    valuenum_col: string
        name of column with the numerical or text variable in question
    label_col: string
        name of column with the lab value label in question
    Returns:
    -----
    filled_summary: df
        list of all variables with # filled values
    missing_summary: df
        list of all variables with # missing values
    percent_summary: df
        list of all variables with # filled values/ # total patients
    """

def missing_fxn(df, valuenum_col, label_col):
    if 'icustay_id' in list(df):
        df= df[df['icustay_id'].isin(list(cohort_df['icustay_id'].unique()))] #good
    else:
        df= df[df['subject_id'].isin(
            list(cohort_df['subject_id'].unique()))] 

    df_notnull = df[df[valuenum_col].notnull()]
    df_null = df[df[valuenum_col].isnull()]
    df_filtered = df.drop_duplicates(['subject_id',label_col])[label_col].value_counts()
    
    filled_summary= df_notnull.drop_duplicates(['subject_id',label_col])[label_col].value_counts()
    missing_summary = df_null.drop_duplicates(['subject_id',label_col])[label_col].value_counts()
    percent_summary = 100*(filled_summary / df_filtered)
    
    #missing_plot=(missing_summary.sort_index().plot.bar(title = f'# of missing values in {df}'))
    
    #filled_summary, missing_summary, percent_summary, 
    return(filled_summary, missing_summary, percent_summary)

In [ ]:
var1, var2, var3,  = missing_fxn(labs_all48, 'valuenum', 'label')
fig, axs = plt.subplots(1,2,figsize=(10,4))
ax1= (var2.sort_index().plot.bar(ax=axs[0], title = '# of missing values in labs_all48'))
ax2= (var3.sort_index().plot.bar(color='grey',
                                ax=axs[1],
                                ylim=(min(var3)-10 ,100),
                                title = '% of values filled in labs_all48'))

#### histogram of frequency over each hour.

In [ ]:
def hours_fxn(td):
    return ((td.days*24*3600) + td.seconds)//3600

def days_fxn(td):
    return td.days

In [ ]:
pd.to_timedelta(vaso_dose_72['delta'].head())

In [ ]:
for element in vaso_dose_72['label'].unique():
    print(element)

In [ ]:
size=11
fig=plt.figure(figsize=(size, size*(2/3)), dpi= 80, facecolor='w')
ax = plt.axes()

vaso_dose_72['hours']=pd.to_timedelta(vaso_dose_72['delta']).map(hours_fxn) #good

for element in vaso_dose_72['label'].unique():    
    (vaso_dose_72.loc[vaso_dose_72.loc[:,'label']==element, 'hours']
         .value_counts(sort=False)
         .sort_index()
         .plot(kind='bar', sort_columns=True))

In [ ]:
for element in vaso_dose_72['label'].unique():
    size=9
    fig=plt.figure(figsize=(size, size*(2/3)), dpi= 80, facecolor='w')
    ax = plt.axes()
    (vaso_dose_72.loc[vaso_dose_72.loc[:,'label']==element, 'hours']
         .value_counts(sort=False)
         .sort_index()
         .plot(kind='bar', sort_columns=True,color='blue'))
    print(element)
    ax.set_ylabel('frequency_for_all_pt', fontsize='large')
    ax.set_xlabel('hours_after_t_0', fontsize='large')
    ax.set_title('%s'%(element), fontsize='large')
    plt.show()

In [ ]:
# #looking at overall missingness

# for element in vaso_dose_72['label'].unique():
#     print(element)
#     print(vaso_dose_72.loc[vaso_dose_72.loc[:,'label']==element, 'icustay_id']
#          .nunique())
#     print(((vaso_dose_72.loc[vaso_dose_72.loc[:,'label']==element, 'icustay_id']
#          .nunique())/len(icustay_id))*100,"% of icustay_id's with any value") 

In [ ]:
# print(vaso_dose_72['icustay_id'].nunique())

In [ ]:
# len(icustay_id)

In [ ]:

11/1/18: After looking at the missingness % of patients without a given reading of each data over first 3 days of icustay, we decided that patients without an single SpO2 reading are disqualified from study. 



 
Total: 14668


10/28/18:

# Goal: to make a new icustay_id cohort with the added criteria that patients have at minimum one Sp02 reading during their first 3 days of suspected infection.
### The idea is that this should be the bare minimum amount of data for a patient, and without it, it's likely the physicians did not suspect an infection in these patients. 

In [ ]:
vitals_all_nosummary_72.head(2)

In [ ]:
#na values
vitals_all_nosummary_72.loc[
    vitals_all_nosummary_72['vitalid'].notnull(),:]#.count() #6930 NULL values

vitals_all_nosummary_nonull= vitals_all_nosummary_72.loc[
    vitals_all_nosummary_72['vitalid'].notnull(),:]#.count() #6930 NULL values


In [ ]:
print(
    vitals_all_nosummary_nonull['icustay_id'].nunique(), #14714/15207 unique icustay_id/icustay_id
)

In [ ]:
vitals_all_nosummary_nonull.loc[vitals_all_nosummary_nonull.loc[:,'vitalid']=='SpO2',:]

#missingness_df.loc[missingness_df.loc[:,'label']=='SpO2',:]

In [ ]:
print((vitals_all_nosummary_nonull.loc[
    vitals_all_nosummary_nonull.loc[:,'vitalid']=='SpO2','icustay_id'
        ].nunique()), #n=14668
    len(icustay_id)
     ) #15207

#print(14668/15207)

icustay_id_vitals = (vitals_all_nosummary_nonull.loc[
    vitals_all_nosummary_nonull.loc[:,'vitalid']=='SpO2','icustay_id'
        ].unique())

In [ ]:

vitals_all_nosummary_nonull_v=(vitals_all_nosummary_nonull.loc[
    vitals_all_nosummary_nonull.loc[:,'icustay_id']
    .isin(icustay_id_vitals),:]
)

In [ ]:
#lists and df with _v will be for patients who have atleast 1 SpO2 reading in first 3 days post t_0. 
vitals_all_nosummary_nonull_v
icustay_id_v= list(vitals_all_nosummary_nonull_v['icustay_id'].unique())
icustay_id_v= [int(x) for x in icustay_id_v]

In [ ]:
#updated final_pt_df2 for patients who have atleast 1 SpO2 reading in first 3 days post t_0. 
final_pt_df2_v = (
    final_pt_df2.loc[final_pt_df2.loc[:,'icustay_id'].isin(icustay_id_v),:]
)

In [ ]:
final_pt_df2_v.head()

In [ ]:
pd.crosstab(final_pt_df2['ab_course'], final_pt_df2['sc_result'])

In [ ]:
#todo, crosstab table of pts in new icustay_id_v
pd.crosstab(final_pt_df2_v['ab_course'], final_pt_df2_v['sc_result']) #n= 14668